In [ ]:
import os
import sys
import shutil
import subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

output_notebook()
plotly.offline.init_notebook_mode()

Make sure you are in the root of `tardis-refdata` after running the following cell.

In [ ]:
cd ..

## Define classes and functions

In [ ]:
# data pickled with protocol 5 can't be opened with `python<3.8.3`, use the backport

if sys.version_info < (3, 8, 3):
    import pickle5

    sys.modules["pickle"] = pickle5

In [ ]:
def highlight_missing(val):
    if val == True:
        return 'background-color: #BCF5A9'
    else:
        return 'background-color: #F5A9A9'
    
def highlight_relative_difference(val):
    ret = 'background-color: #BCF5A9'
    if val is None:
        ret = 'background-color: #BCF5A9'
    elif val > 1e-2:
        ret = 'background-color: #F2F5A9'
    elif val > 1e-1:
        ret = 'background-color: #F5D0A9'
    elif val > 1:
        ret = 'background-color: #F5A9A9'
    return ret

In [ ]:
class ReferenceComparerFromPaths(object):
    def __init__(self, ref1_path, ref2_path, compare_path='unit_test_data.h5'):
        self.ref1_fname = ref1_path
        self.ref2_fname = ref2_path
        self.compare_path = compare_path
        
    def generate_test_table(self):
        rd1_hdfs = pd.HDFStore(self.ref1_fname, mode='r')
        rd2_hdfs = pd.HDFStore(self.ref2_fname, mode='r')
        rd1_keys = rd1_hdfs.keys()
        rd2_keys = rd2_hdfs.keys()
        rd1_hdfs.close()
        rd2_hdfs.close()
        rd1_df = pd.DataFrame(index=rd1_keys, columns=['exists'])
        rd2_df = pd.DataFrame(index=rd2_keys, columns=['exists'])
        rd1_df['exists'] = True
        rd2_df['exists'] = True
        joined_df = rd1_df.join(rd2_df, how='outer', lsuffix='_1', rsuffix='_2')
        joined_df = joined_df.fillna(False)
        return joined_df
    
    def compare_refdata(self, test_table):
        test_table['match'] = None
        test_table['abs_diff_mean'] = None
        test_table['abs_diff_max'] = None
        test_table['rel_diff_mean'] = None
        test_table['rel_diff_max'] = None
        for row_id, row in test_table.iterrows():
            
            if row[['exists_1', 'exists_2']].all():
                ref1_df = pd.read_hdf(self.ref1_fname, row_id)
                ref2_df = pd.read_hdf(self.ref2_fname, row_id)
                
                if isinstance(ref1_df, pd.Series):
                    try:
                        pd.testing.assert_series_equal(ref1_df, ref2_df)
                    except AssertionError:
                        test_table.loc[row_id, 'match'] = False
                        abs_diff = np.fabs(ref1_df - ref2_df)
                        rel_diff = (abs_diff / np.fabs(ref1_df))[ref1_df != 0]
                        test_table.loc[row_id, 'abs_diff_mean'] = abs_diff.mean()
                        test_table.loc[row_id, 'abs_diff_max'] = abs_diff.max()
                        test_table.loc[row_id, 'rel_diff_mean'] = rel_diff.mean()
                        test_table.loc[row_id, 'rel_diff_max'] = rel_diff.max()
                    else:
                        test_table.loc[row_id, 'match'] = True

                elif isinstance(ref1_df, pd.DataFrame):
                    try:
                        pd.testing.assert_frame_equal(ref1_df, ref2_df)
                    except AssertionError:
                        test_table.loc[row_id, 'match'] = False
                        abs_diff = np.fabs(ref1_df - ref2_df)
                        rel_diff = (abs_diff / np.fabs(ref1_df))[ref1_df != 0]
                        test_table.loc[row_id, 'abs_diff_mean'] = abs_diff.mean(skipna=True).mean()
                        test_table.loc[row_id, 'abs_diff_max'] = abs_diff.max(skipna=True).max()
                        test_table.loc[row_id, 'rel_diff_mean'] = rel_diff.mean(skipna=True).mean()
                        test_table.loc[row_id, 'rel_diff_max'] = rel_diff.max(skipna=True).max()
                    else:
                        test_table.loc[row_id, 'match'] = True

                else:
                    raise ValueError('Needs to be a Series or DataFrame but is' + str(type(ref1_df)))
        return test_table
        
    def generate_test_table_from_direct_paths(self):
        tt = self.generate_test_table()
        tt = self.compare_refdata(tt)
        return tt

## Load data

In [ ]:
REF1_PATH = os.environ['REF1_PATH']

if not REF1_PATH:
    raise ValueError

REF2_PATH = os.environ['REF2_PATH']

if not REF2_PATH:
    raise ValueError

In [ ]:
# REF1_PATH="/Users/atharva/workspace/code/tardis-main/tardis-refdata/unit_test_data.h5"
# REF2_PATH="/Users/atharva/workspace/code/tardis-main/tardis-refdata2/unit_test_data.h5"
REF1_PATH, REF2_PATH

## Test table

In [ ]:
comparer = ReferenceComparerFromPaths(
    ref1_path=REF1_PATH, 
    ref2_path=REF2_PATH, 
)
tt = comparer.generate_test_table_from_direct_paths()

In [ ]:
tt[["exists_1", "exists_2", 'rel_diff_mean', 'rel_diff_max', 'match']].style.applymap(
    highlight_missing, subset=['exists_1', 'exists_2', 'match']).applymap(
    highlight_relative_difference, subset=['rel_diff_mean', 'rel_diff_max'])

## Detailed inspection of the reference data

If parts of the reference data show differences between revisions, you should invest some time examining these differences in detail. Often, visualizing the relevant data blocks already helps. 

You can use the following plotting routines as a blueprint and adjust and extend them to your needs.

In [ ]:
def compare_output_nu(df1, df2, mpl_backend=False):
    nu_min = np.min([df1.min(), df2.min()])
    nu_max = np.max([df1.max(), df2.max()])
    
    fig = make_subplots(rows=1, cols=2)
    
    fig.add_trace(
        go.Scatter(x=df1, y=df2, mode="markers", marker=dict(size=3)), row=1, col=1
    )
    fig.update_xaxes(title_text="output_nu, ref 1", row=1, col=1)
    fig.update_yaxes(title_text="output_nu, ref 2", row=1, col=1)
    
    fig.add_trace(
        go.Histogram(
            x=df1,
            nbinsx=100,
            histfunc="count",
            name="ref 1",
            marker=dict(color='rgba(0,0,0,0)', line=dict(width=1, color='orange')),
            opacity=0.5,
            xbins=dict(start=nu_min, end=nu_max, size=(nu_max - nu_min) / 100),
        ),
        row=1,
        col=2,
    )
    fig.add_trace(
        go.Histogram(
            x=df2,
            nbinsx=100,
            histfunc="count",
            name="ref 2",
            marker=dict(color='rgba(0,0,0,0)', line=dict(width=1, color='blue')),
            xbins=dict(start=nu_min, end=nu_max, size=(nu_max - nu_min) / 100),
            opacity=0.5,
        ),
        row=1,
        col=2,
    )
    fig.update_layout(barmode="overlay", height=550)  # Overlay the histograms
    fig.update_xaxes(title_text="output_nu", row=1, col=2)
    
    fig.show()

In [ ]:
def compare_spectrum(ref1_nu, ref1_lam, ref1_L, ref2_nu, ref2_lam, ref2_L, mpl_backend=False):
    fig = make_subplots(rows=2, cols=2,)
    
    fig.add_trace(go.Scatter(x=ref1_nu, y=ref1_L, mode='lines', name='ref 1', marker=dict(size=2)), row=1, col=1)
    fig.add_trace(go.Scatter(x=ref2_nu, y=ref2_L, mode='lines', name='ref 2', marker=dict(size=2)), row=1, col=1)
    
    fig.add_trace(go.Scatter(x=ref1_nu, y=ref1_L / ref2_L, mode='markers', name='L ref 1 / L ref 2', marker=dict(size=2)), row=1, col=2)
    
    fig.add_trace(go.Scatter(x=ref1_lam, y=ref1_L, mode='lines', name='ref 1', marker=dict(size=2)), row=2, col=1)
    fig.add_trace(go.Scatter(x=ref2_lam, y=ref2_L, mode='lines', name='ref 2', marker=dict(size=2)), row=2, col=1)
    
    fig.add_trace(go.Scatter(x=ref1_lam, y=ref1_L / ref2_L, mode='markers', name='L ref 1 / L ref 2', marker=dict(size=2)), row=2, col=2)
    
    fig.update_xaxes(title_text="nu", row=1, col=1)
    fig.update_yaxes(title_text="L", row=1, col=1)
    fig.update_xaxes(title_text="nu", row=1, col=2)
    fig.update_yaxes(title_text="L ref 1 / L ref 2", row=1, col=2)
    fig.update_xaxes(title_text="lambda", row=2, col=1)
    fig.update_yaxes(title_text="L", row=2, col=1)
    fig.update_xaxes(title_text="lambda", row=2, col=2)
    fig.update_yaxes(title_text="L ref 1 / L ref 2", row=2, col=2)
    fig.update_layout(height=1000)
    
    fig.show()

Get the data and find all the entries for which differences exist:

In [ ]:
tmp1 = pd.HDFStore(comparer.ref1_fname, "r")
tmp2 = pd.HDFStore(comparer.ref2_fname, "r")

diff_entries = tt.loc[(tt["match"] == False) & (tt["exists_1"] == True) & (tt["exists_2"] == True)].index

## Results

In [ ]:
compare_output_nu(tmp1['/test_simulation/output_nu'], tmp2['/test_simulation/output_nu'])

In [ ]:
compare_spectrum(tmp1['/test_transport_simple/spectrum/_frequency'][:-1], 
                 tmp1['/test_transport_simple/spectrum/wavelength'],
                 tmp1['/test_transport_simple/spectrum/luminosity'],
                 tmp2['/test_transport_simple/spectrum/_frequency'][:-1], 
                 tmp2['/test_transport_simple/spectrum/wavelength'],
                 tmp2['/test_transport_simple/spectrum/luminosity'])

In [ ]:
compare_spectrum(tmp1['/test_transport_simple_integral_macroatom_interp/spectrum/_frequency'][:-1], 
                 tmp1['/test_transport_simple_integral_macroatom_interp/spectrum/wavelength'],
                 tmp1['/test_transport_simple_integral_macroatom_interp/spectrum_integrated/luminosity'],
                 tmp2['/test_transport_simple_integral_macroatom_interp/spectrum/_frequency'][:-1], 
                 tmp2['/test_transport_simple_integral_macroatom_interp/spectrum/wavelength'],
                 tmp2['/test_transport_simple_integral_macroatom_interp/spectrum_integrated/luminosity'])

In [ ]:
compare_spectrum(tmp1['/test_transport_simple_integral_macroatom/spectrum/_frequency'][:-1], 
                 tmp1['/test_transport_simple_integral_macroatom/spectrum_integrated/wavelength'],
                 tmp1['/test_transport_simple_integral_macroatom/spectrum_integrated/luminosity'],
                 tmp2['/test_transport_simple_integral_macroatom/spectrum/_frequency'][:-1], 
                 tmp2['/test_transport_simple_integral_macroatom/spectrum_integrated/wavelength'],
                 tmp2['/test_transport_simple_integral_macroatom/spectrum_integrated/luminosity'])

In [ ]:
compare_spectrum(tmp1['/test_transport_simple_integral_downbranch/spectrum/_frequency'][:-1], 
                 tmp1['/test_transport_simple_integral_downbranch/spectrum/wavelength'],
                 tmp1['/test_transport_simple_integral_downbranch/spectrum_integrated/luminosity'],
                 tmp2['/test_transport_simple_integral_downbranch/spectrum/_frequency'][:-1], 
                 tmp2['/test_transport_simple_integral_downbranch/spectrum/wavelength'],
                 tmp2['/test_transport_simple_integral_downbranch/spectrum_integrated/luminosity'])

In [ ]:
compare_spectrum(tmp1['/test_transport_simple/spectrum_virtual/_frequency'][:-1], 
                 tmp1['/test_transport_simple/spectrum_virtual/wavelength'],
                 tmp1['/test_transport_simple/spectrum_virtual/luminosity'],
                 tmp2['/test_transport_simple/spectrum_virtual/_frequency'][:-1], 
                 tmp2['/test_transport_simple/spectrum_virtual/wavelength'],
                 tmp2['/test_transport_simple/spectrum_virtual/luminosity'])

In [ ]:
if False in tt.match.values or None in tt.match.values:
    print("Reference files do not match, setting contents of tmp file to zero.")
    with open('refdata_compare_result', 'w+') as fh:
        fh.write('REFDATA COMPARISON FAILED')
else:
    print("Reference files match.")
    with open('refdata_compare_result', 'w+') as fh:
        fh.write('REFDATA COMPARISON SUCCEEDED')   